In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pprint

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('loan.csv') #save data to a pandas dataframe

In [3]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

# This chunk gets the data used for the proposal

In [4]:
usefulColumns = ['total']
bankIncomeColumns = ['recoveries', 'collection_recovery_fee', ]

#we have loan amount and total payment, maybe use these
# benefit of lending a loan for the bank = avg interest paid (total_rec_int) + avg late payments recived (tatal_rec_late_fee)
benies = df['total_rec_int'].mean() + df['total_rec_late_fee'].mean()
benies

1755.2010184318683

In [5]:
# cost of defaulting on a loan = avg loan amount (loan_amnt)
    #- avg recoveries (recoveries) + avg collection recovery fee (collection_recovery_fee)
d = (df['loan_amnt'] - df['total_pymnt'])
defaultCost = d.mean() - df['recoveries'].mean() + df['collection_recovery_fee'].mean()
defaultCost

7155.399435454788

# Cost Matrix

In [6]:

#                  predicted default           predicted paid
# real default     0                           14714
#
# real paid        1755                        -1755
# wrong = 1755 + 14,714 = 16,469
# correct = - 1755

# This chunk will clean the data into clean_loan.csv

In [7]:
df['id']

0          1077501
1          1077430
2          1077175
3          1076863
4          1075358
            ...   
887374    36371250
887375    36441262
887376    36271333
887377    36490806
887378    36271262
Name: id, Length: 887379, dtype: int64

In [8]:
useful_columns = []
#don't understand: loan_amnt vs funded_amnt vs funded_amnt_inv
useful_columns = ['id', 'loan_amnt']
print(df.id.count()) #total number of rows

#how many rows have loan amnt != funded_amnt?
df[df.loan_amnt != df.funded_amnt].id

887379


61        1068934
66        1069093
70        1069043
77        1068416
80        1068994
           ...   
230644    1042487
230646    1063649
230647    1062754
230650    1062334
230656    1058291
Name: id, Length: 2062, dtype: int64

In [9]:
df[df.loan_amnt < df.funded_amnt].id # => all loans are >= funded_amnt

Series([], Name: id, dtype: int64)

I have no idea what the difference/significance is of loan_amnt vs funded_amnt_ vd funded_amnt_inv. I will just use loan_amnt. I believe they are not useful

In [10]:
# term may be useful for a decision tree
# int_rate is included. Lower interest rate => better credit score => likey to pay off?
# installment is included bc a lower payment is easier to pay => no default?
# grade and sub_grade are not included bc it just determines the interest rate see:
#                  https://www.lendingclub.com/foliofn/rateDetail.action
# emp_title is the employment title of borrower, not included bc a loan can be defaulted by anyone
# emp_length included bc long term employment => no default
# home_ownership included and will be transfomed into a boolean (own home vs not) bc home owners have collateral
# annual_inc (annual income) replaced by dti
# verification_status included, will be changed to boolean (people lie about cash flow)
# issue_d - the month the loan was funded was left out 
####### loan_status (Fully Paid vs Charged Off) is defaulted vs not########
# 

useful_columns.append(['term', 'int_rate', 'installment', 'emp_length', 'home_ownership', \
                     'verification_status', 'loan_status'])

Should pymnt_plan be included? payment plan => no default??

In [11]:
# how many, out of the total entries, have a payment plan?
print('Out of ' + str(df.id.count()) + ' rows, '  + str(df[df.pymnt_plan != 'n'].id.count()) + ' have payment plan')

Out of 887379 rows, 10 have payment plan


pymnt_plan data is super skewed, will not use

In [12]:
# url takes you to the lending club website
# desc is a description of why the loan was gotten, left out bc no (reasonable) way to turn into numbers
# purpose similar to desc
# title similar to desc
# zip_code (may need to be changed to float) used bc first 3 numbers tell what region borrower is in
# addr_state left out bc that infor is in zip_code
# dti is debt to income ratio
# deling_2yrs is the number of times borrower has been deliquent on any reported line of credit in 2yrs
# earliest_cr_line used due to data type and hopefully doesnt hurt anything
# inq_last_6mths used bc more inquiries => default
# 
useful_columns.append(['zip_code', 'dti', 'delinq_2yrs', 'inq_last_6mths'])

In [13]:
# mths_since_last_delinq included if full enough bc longer time => no default
df['mths_since_last_delinq']
# is the column at least 75% full?
if((df['mths_since_last_delinq'].dropna().count()) > ((3/4) * (df['mths_since_last_delinq'].count()))):
    print('mths_since_last_delinq is at least 75% full')
else:
    print('mths_since_last_delinq is not at least 75% full')
df['mths_since_last_delinq'].dropna()

mths_since_last_delinq is at least 75% full


3         35.0
4         38.0
16        61.0
18         8.0
27        20.0
          ... 
887370    69.0
887371    65.0
887372     9.0
887375    26.0
887377    22.0
Name: mths_since_last_delinq, Length: 433067, dtype: float64

In [14]:
# mths_since_last_record is months since last public record. Not sure record of what so not included
# open_acc is the number of open credit lines in borrowers name, inluded bc more => no default 
# pub_rec is the number of derogetory marks on public record included bc less marks => on time payments
# revol_bal is the total credit revolving balance not included bc revol_util is included
# revol_util is the amount of credit used relative to total available
# total_acc is total credit lines on file (so including paid off ones) may be used to create new col with open_acc
# initial_list_status is w (whole) or f(fractional) since it can be converted to binary its kept
useful_columns.append(['open_acc', 'pub_rec', 'revol_util', 'total_acc'] )

# how many initial_list_status are f?
print(df[df['initial_list_status'] == 'f'].count())
print(df.initial_list_status.count())
# so about half => not super skewed

id                  456848
member_id           456848
loan_amnt           456848
funded_amnt         456848
funded_amnt_inv     456848
                     ...  
all_util              2645
total_rev_hi_lim    386572
inq_fi                2645
total_cu_tl           2645
inq_last_12m          2645
Length: 74, dtype: int64
887379


In [15]:
#out_prncp amount left
useful_columns.append(['initial_list_status', 'out_prncp'])

# how many rows have 0 in out_prncp?
(df[df['out_prncp'] == 0].count()) / (df['out_prncp'].count())
# so about 29% of rows that have out_prncp are 0 => enough data to keep

id                  0.288262
member_id           0.288262
loan_amnt           0.288262
funded_amnt         0.288262
funded_amnt_inv     0.288262
                      ...   
all_util            0.000165
total_rev_hi_lim    0.213371
inq_fi              0.000165
total_cu_tl         0.000165
inq_last_12m        0.000165
Length: 74, dtype: float64

In [16]:
# out_prncp_inv is amount remaining on loan by investors. I don't understand the use so not included
# total_pymnt is payments recived to date for total amnt funded
#total_pymnt_inv is Payments received to date for portion of total amount funded by investors
# total_rec_prncp is used instead of total_pymnt and total_pymnt_inv
# total_rec_int is Interest received to date
# total_rec_late_fee is Late fees received to date
#######4 previous columns should be combined into total money made##############
# 
# recoveries post charge off gross recovery (amnt gained from defaulted loan) used for cost matrix
# collection_recovery_fee post charge off collection fee used for cost matrix
# last_pymnt_d is Last month payment was received not used 
# last_pymnt_amnt is Last total payment amount received not used
# next_pymnt_d is Next scheduled payment date
# last_credit_pull_d is The most recent month LC pulled credit for this loan not used
# collections_12_mths_ex_med is Number of collections in 12 months excluding medical collections used
# mths_since_last_major_derog is Months since most recent 90-day or worse rating used 
# policy_code is "publicly available policy_code=1 new products not publicly available policy_code=2" not used
# application_type is Indicates whether the loan is an individual application or a joint application with two co-borrowers used 
                # bc more people => no default
# annual_inc_joint is The combined self-reported annual income provided by the co-borrowers during registration (in not joint, no data) \
            # replaced with dti_joint
# dti_joint is A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, 
            # excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income. use instead of annual_inc
useful_columns.append(['total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',\
                       'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'application_type', 'dti_joint'])
useful_columns
#type(useful_columns)

['id',
 'loan_amnt',
 ['term',
  'int_rate',
  'installment',
  'emp_length',
  'home_ownership',
  'verification_status',
  'loan_status'],
 ['zip_code', 'dti', 'delinq_2yrs', 'inq_last_6mths'],
 ['open_acc', 'pub_rec', 'revol_util', 'total_acc'],
 ['initial_list_status', 'out_prncp'],
 ['total_rec_prncp',
  'total_rec_int',
  'total_rec_late_fee',
  'recoveries',
  'collection_recovery_fee',
  'collections_12_mths_ex_med',
  'mths_since_last_major_derog',
  'application_type',
  'dti_joint']]

In [17]:
# verification_status_joint Indicates if the co-borrowers' joint income was verified by LC,/
            # not verified, or if the income source was verified used bc can be turned numerical
# acc_now_delinq is The number of accounts on which the borrower is now delinquent. used bc higher => default
# tot_coll_amt is Total collection amounts ever owed #### can maybe replace the collection columns but not sure yet
# tot_cur_bal is Total current balance of all accounts not needed bc dti is included 
# open_acc_6m is Number of open trades in last 6 months
# open_il_6m is Number of currently active installment trades used 
# open_il_12m is Number of installment accounts opened in past 12 months
# open_il_24m is Number of installment accounts opened in past 24 months 
useful_columns.append(['verification_status_joint', 'acc_now_delinq'])


In [18]:
def enough_num_data(col, data):
    return str((len(df[df[col] > 0]) / len(df[col])) * 100) +'%'

In [19]:
# enough data in tot_coll_amnt?
print(enough_num_data('tot_coll_amt', df) + ' of tot_coll_amt is nonzero') # not useful anyway
print(enough_num_data('open_acc_6m', df) + ' of open_acc_6m is nonzero')
print(enough_num_data('open_il_6m', df) + ' of open_acc_6m is nonzero')
print(enough_num_data('open_il_12m', df) + ' of open_acc_12m is nonzero')
print(enough_num_data('open_il_24m', df) + ' of open_acc_24m is nonzero')
print(enough_num_data('mths_since_rcnt_il', df) + ' of mths_since_rcnt_il is nonzero')
print(enough_num_data('total_bal_il', df) + ' of total_bal_il is nonzero')
print(enough_num_data('il_util', df) + ' of il_util is nonzero')
print(enough_num_data('open_rv_12m', df) + ' of open_rv_12m is nonzero')
print(enough_num_data('open_rv_24m', df) + ' of open_rv_24m is nonzero')
print(enough_num_data('max_bal_bc', df) + ' of max_bal_bc is nonzero')
print(enough_num_data('all_util', df) + ' of all_util is nonzero')
print(enough_num_data('total_rev_hi_lim', df) + ' of total_rev_hi_lim is nonzero') ##########
print(enough_num_data('inq_fi', df) + ' of inq_fi is nonzero')
print(enough_num_data('total_cu_tl', df) + ' of total_cu_tl is nonzero')

13.089897326846813% of tot_coll_amt is nonzero
1.4791875850115903% of open_acc_6m is nonzero
2.147673091204547% of open_acc_6m is nonzero
1.1906975486235305% of open_acc_12m is nonzero
1.7907793625947874% of open_acc_24m is nonzero
2.3436434713915926% of mths_since_rcnt_il is nonzero
2.153871119330072% of total_bal_il is nonzero
2.0824247587558418% of il_util is nonzero
1.6113746212159628% of open_rv_12m is nonzero
2.085242044267444% of open_rv_24m is nonzero
2.3624629386090947% of max_bal_bc is nonzero
2.406863358271945% of all_util is nonzero
92.0481552977927% of total_rev_hi_lim is nonzero
1.1225192392427588% of inq_fi is nonzero
1.114968914071665% of total_cu_tl is nonzero


In [20]:
type(useful_columns[0])
#type((useful_columns[2]))

str

In [21]:
useful_columns

['id',
 'loan_amnt',
 ['term',
  'int_rate',
  'installment',
  'emp_length',
  'home_ownership',
  'verification_status',
  'loan_status'],
 ['zip_code', 'dti', 'delinq_2yrs', 'inq_last_6mths'],
 ['open_acc', 'pub_rec', 'revol_util', 'total_acc'],
 ['initial_list_status', 'out_prncp'],
 ['total_rec_prncp',
  'total_rec_int',
  'total_rec_late_fee',
  'recoveries',
  'collection_recovery_fee',
  'collections_12_mths_ex_med',
  'mths_since_last_major_derog',
  'application_type',
  'dti_joint'],
 ['verification_status_joint', 'acc_now_delinq']]

In [22]:
# the only col with enough data is total_rev_hi_lim whic is Total revolving high credit/credit limit
useful_columns.append('total_rev_hi_lim')
used_cols = []
for x in useful_columns:
    if(type(x) == str):
        used_cols.append(x)
    else:
        for y in x:
             used_cols.append(y) 
used_cols
len(df.columns)

74

In [23]:

used_df = df[used_cols].copy()
used_df.head(5)

,id,loan_amnt,term,int_rate,installment,emp_length,home_ownership,verification_status,loan_status,zip_code,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,dti_joint,verification_status_joint,acc_now_delinq,total_rev_hi_lim
0,1077501,5000.0,36 months,10.65,162.87,10+ years,RENT,Verified,Fully Paid,860xx,...,0.00,0.00,0.00,0.0,NaN,INDIVIDUAL,NaN,NaN,0.0,NaN
1,1077430,2500.0,60 months,15.27,59.83,< 1 year,RENT,Source Verified,Charged Off,309xx,...,0.00,117.08,1.11,0.0,NaN,INDIVIDUAL,NaN,NaN,0.0,NaN
2,1077175,2400.0,36 months,15.96,84.33,10+ years,RENT,Not Verified,Fully Paid,606xx,...,0.00,0.00,0.00,0.0,NaN,INDIVIDUAL,NaN,NaN,0.0,NaN
3,1076863,10000.0,36 months,13.49,339.31,10+ years,RENT,Source Verified,Fully Paid,917xx,...,16.97,0.00,0.00,0.0,NaN,INDIVIDUAL,NaN,NaN,0.0,NaN
4,1075358,3000.0,60 months,12.69,67.79,1 year,RENT,Source Verified,Current,972xx,...,0.00,0.00,0.00,0.0,NaN,INDIVIDUAL,NaN,NaN,0.0,NaN


In [24]:
used_df.to_csv("used_df.csv")

# NEXT STEPS
- make all columns numerical
- combine/ create new columns
- write ML program

# Clustering

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.simplefilter("ignore", UserWarning)

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [26]:
sse = [] # Sum of Squared Error of data points to centroid; The smaller the better.
for i in range(1, 20): # get the prefect number of clusters from 1 to 20
    customer_kmeans = KMeans(n_clusters = i, random_state = 42)
    customer_kmeans.fit(used_df)
    sse.append(customer_kmeans.inertia_)
    
plt.plot(range(1, 20), sse, marker = 'o')
plt.plot(4, sse[3], color = 'r', marker = 'o') # plot red mark on k = 4
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

ValueError: could not convert string to float: ' 36 months'